## Procesamiento inicial de los ficheros de audio

En esta parte lo único que voy a hacer va a ser transformar el fichero de auido a **.mp3**. Para todo lo que es el procesamiento de los audios, se usará la librería [pydub](https://pydub.com)

In [5]:
pip install --upgrade pip 

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
#Checkeo que se encuentra el archivo
os.path.exists("./audios/audio_prueba.ogg")

True

In [8]:
from pydub import AudioSegment
sound = AudioSegment.from_file("./audios/audio_prueba.ogg") #Puedes meter aquí cualquier tipo de audio

sound.export("./audios/salida/audio_prueba.mp3", format="mp3", bitrate="128k")

<_io.BufferedRandom name='./audios/salida/audio_prueba.mp3'>

## Uso del modelo



In [9]:
pip install --upgrade transformers datasets[audio] accelerate

Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
print("PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

PyTorch: 2.5.1+cu121
CUDA disponible: True
GPU: NVIDIA GeForce RTX 4060


In [12]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe("./audios/salida/audio_prueba.mp3", return_timestamps=True)


Device set to use cuda:0
F:\Programas\MiniConda\envs\tfg\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [13]:
with open("./salida/salida_unico_archivo_whisper_large_turbo.txt", "w") as archivo:
    archivo.write(result["text"])

## Resultados de la prueba

Ultima ejecución día 3 de junio de 2025 ha tardado 1403 segundos